# Extract images features

In [1]:
from google.colab import drive
import sys
import os

drive.mount('/content/drive')

sys.path.append(os.path.abspath('/content/drive/MyDrive/Università/NAML_project/Utilities'))

Mounted at /content/drive


## Utility functions

In [2]:
from skimage.feature import local_binary_pattern
from PIL import Image
import cv2
import numpy as np
import os
import shutil

def lbp_concatenate(path):
    """
    Perform Local Binary Patterns (LBP) on the given image two times with different parameters
    and concatenate the results.

    Args:
        image (numpy.ndarray): Input image.

    Returns:
        numpy.ndarray: Concatenated LBP results.
    """
    # Open the image and convert it to grayscale
    image = jpg_to_grayscale(path)

    # Perform LBP with parameters R=1 and P=8
    lbp1 = local_binary_pattern(image, P=8, R=1, method='uniform')

    # Perform LBP with parameters R=2 and P=16
    lbp2 = local_binary_pattern(image, P=16, R=2, method='uniform')

    # Compute histogram1
    hist1, _ = np.histogram(lbp1.ravel(), bins=np.arange(0, 8 + 3), range=(0, 8 + 2))

    # Normalize the histogram
    hist1 = hist1.astype("float")
    hist1 /= (hist1.sum() + 1e-7)

    # Compute histogram2
    hist2, _ = np.histogram(lbp1.ravel(), bins=np.arange(0, 8 + 3), range=(0, 8 + 2))

    # Normalize the histogram
    hist2 = hist2.astype("float")
    hist2 /= (hist2.sum() + 1e-7)

    # Concatenate the results
    concatenated_lbp = np.concatenate((hist1, hist2))

    return concatenated_lbp


def jpg_to_grayscale(image_path):
    """
    Opens a JPEG image file as a numpy ndarray and converts it to grayscale.

    Args:
        image_path (str): Path to the JPEG image file.

    Returns:
        numpy.ndarray: Grayscale image as a numpy ndarray.
    """
    # Open the JPEG image file
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return gray


def print_lbp_histogram(concatenated_lbp, num_points):
    """
    Compute and print the histogram of concatenated LBP results.

    Args:
        concatenated_lbp (numpy.ndarray): Concatenated LBP results.
    """
    # Compute the histogram of concatenated LBP results
    hist, bins = np.histogram(concatenated_lbp, bins=np.arange(0, num_points + 3),
			range=(0, num_points + 2))

    # Print the histogram
    for i, count in enumerate(hist):
        print(f"Bin {bins[i]} - {bins[i+1]}: {count}")


# Utility function to move subfolders to parent folder
def move_subfolders_to_parent(folder_path):
  # Iterate through subdirectories of folder2
  for root, dirs, files in os.walk(folder_path):
    for subdir in dirs:
      subdir_path = os.path.join(root, subdir)
      # Move each subdirectory to folder1
      shutil.move(subdir_path, os.path.dirname(folder_path))


# Utility function to delete directories
def delete_directory(directory):
  """Deletes a directory and all its contents.

  Args:
      directory (str): The path to the directory to be deleted.
  """
  print('Deleting directory ' + directory)
  try:
    shutil.rmtree(directory)
    print(f"Directory '{directory}' and all its contents have been successfully deleted.")
  except Exception as e:
    print(f"An error occurred while deleting the directory '{directory}': {e}")

## Extract features from images

In [3]:
import csv
import LPQ, RICLBP

datasets = [
    # 'gtzan_aim',
    # 'homburg_aim',
    'ismir2004_aim']

# datasets = [
#     'gtzan_spectrum',
#     'homburg_spectrum',
#     'ismir2004_spectrum']
datasets_drive_path = '/content/drive/MyDrive/Università/NAML_project/datasets_processed/'

for dataset in datasets:
  # upload and unzip the dataset from google drive
  dataset_folder = datasets_drive_path + dataset + '.zip'
  !unzip "$dataset_folder"

  move_subfolders_to_parent('/content/content')

  # Create csv file to save feature value
  csv_path = datasets_drive_path + dataset + '_texture_features.csv'
  header = ['label', 'LBP', 'LBPHF', 'RICLBP',
            # 'DSLBP',
            'LPQ']
  # with open(csv_path, 'w', newline='') as csvfile:
  #       writer = csv.writer(csvfile)
  #       writer.writerow(header)

  # Iterate throught the genres in the dataset
  # genres = os.listdir('/content/' + dataset)
  genres = [
      # 'pop',
      # 'classical',
      # 'metal',
      # 'electronic',
      'world',
      'rock',
      'jazz',
      'punk'
      ]
  for genre in genres:
    # Iterate throught the images for each genre
    imgs = os.listdir('/content/' + dataset + '/' + genre)
    count = 0
    for img in imgs:
      if genre != 'world' or count > 3196:
        img_path = '/content/' + dataset + '/' + genre + '/' + img

        # Initilize feature list with target label
        features = [genre]

        # Compute lbp histogram
        lbp = lbp_concatenate(img_path)

        # Add lbp to features list
        features.append(lbp)

        # Compute lbphf
        lbphf = np.fft.fft(lbp)

        # Add lbphf to features list
        features.append(lbphf)

        # Compute riclbp
        img = jpg_to_grayscale(img_path)
        riclbp = RICLBP.getfeatures(img)

        # Add riclbp to features list
        features.append(riclbp)

        # TODO: capire come fare DSLBP

        # Compute lpq
        lpq = LPQ.lpq(img_path, mode="nh")

        # Add lpq to features list
        features.append(lpq)

        # Save features
        with open(csv_path, 'a', newline='') as csvfile:
          writer = csv.writer(csvfile)
          writer.writerow(features)

        # Log
        print('Processed file ' + img_path)

      if genre == 'world' and count < 657:
        print('increase count')
        count += 1

  delete_directory('/content/' + dataset)

Output streaming troncato alle ultime 5000 righe.
  inflating: ismir2004_aim/world/5-go-12.wav.png  
  inflating: ismir2004_aim/world/5-go-13.wav.png  
  inflating: ismir2004_aim/world/5-go-14.wav.png  
  inflating: ismir2004_aim/world/5-go-15.wav.png  
  inflating: ismir2004_aim/world/5-go-16.wav.png  
  inflating: ismir2004_aim/world/5-go-17.wav.png  
  inflating: ismir2004_aim/world/5-go-18.wav.png  
  inflating: ismir2004_aim/world/5-go-19.wav.png  
  inflating: ismir2004_aim/world/5-go-2.wav.png  
  inflating: ismir2004_aim/world/5-go-20.wav.png  
  inflating: ismir2004_aim/world/5-go-21.wav.png  
  inflating: ismir2004_aim/world/5-go-22.wav.png  
  inflating: ismir2004_aim/world/5-go-23.wav.png  
  inflating: ismir2004_aim/world/5-go-24.wav.png  
  inflating: ismir2004_aim/world/5-go-25.wav.png  
  inflating: ismir2004_aim/world/5-go-26.wav.png  
  inflating: ismir2004_aim/world/5-go-27.wav.png  
  inflating: ismir2004_aim/world/5-go-28.wav.png  
  inflating: ismir2004_aim/world/

In [ ]:
delete_directory('ismir2004')

Deleting directory ismir2004
Directory 'ismir2004' and all its contents have been successfully deleted.


In [ ]:
import LPQ, RICLBP

datasets_drive_path = '/content/drive/MyDrive/Università/NAML_project/datasets_processed/'
dataset_folder = datasets_drive_path + 'ismir2004' + '.zip'
!unzip "$dataset_folder"

move_subfolders_to_parent('/content/content')



Output streaming troncato alle ultime 5000 righe.
  inflating: content/ismir2004/jazz/8-down_the_line-3.wav  
  inflating: content/ismir2004/jazz/1-like_white_on_rice-6.wav  
  inflating: content/ismir2004/jazz/10-abbey_rhodes-14.wav  
  inflating: content/ismir2004/jazz/12-hopscotch-6.wav  
  inflating: content/ismir2004/jazz/5-albuterol-23.wav  
  inflating: content/ismir2004/jazz/9-silk_road-24.wav  
  inflating: content/ismir2004/jazz/14-preachin'_blues-11.wav  
  inflating: content/ismir2004/jazz/19-sittin'_under_the_tree-10.wav  
  inflating: content/ismir2004/jazz/19-cause_you_been_messin_'round-6.wav  
  inflating: content/ismir2004/jazz/1-prospects-12.wav  
  inflating: content/ismir2004/jazz/7-claytonia-6.wav  
  inflating: content/ismir2004/jazz/5-albuterol-9.wav  
  inflating: content/ismir2004/jazz/7-claytonia-15.wav  
  inflating: content/ismir2004/jazz/3-you_think_too_much-20.wav  
  inflating: content/ismir2004/jazz/6-oaks-2.wav  
  inflating: content/ismir2004/jazz/20-

TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
genres = []

for genre in os.listdir('/content/ismir2004'):
  genres.append(genre)
  count = 0
  for i in os.listdir('/content/ismir2004/' + genre):
    count += 1
  print(genre + ' : ' + str(count))

print(genres)

pop : 162
classical : 5448
metal : 850
electronic : 3715
world : 4227
rock : 2065
jazz : 584
punk : 256
['pop', 'classical', 'metal', 'electronic', 'world', 'rock', 'jazz', 'punk']


In [ ]:
datasets = ['gtzan_spectrum']
datasets_drive_path = '/content/drive/MyDrive/Università/NAML_project/datasets_processed/'

for dataset in datasets:
  # upload and unzip the dataset from google drive
  dataset_folder = datasets_drive_path + dataset + '.zip'
  !unzip "$dataset_folder"

  move_subfolders_to_parent('/content/content')

Archive:  /content/drive/MyDrive/Università/NAML_project/datasets_processed/gtzan_spectrum.zip
  inflating: content/gtzan_spectrum/hiphop/hiphop.00039-1.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00046-1.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00046-2.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00086-0.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00084-1.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00077-0.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00020-1.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00034-1.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00006-0.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00035-0.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00082-0.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00019-0.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00031-1.jpg  
  inflating: content/gtzan_spectrum/hiphop/hiphop.00067-0.jpg  
  inflat

In [ ]:
print(len(os.listdir('/content/gtzan_spectrum/rock')))

299
